<a href="https://colab.research.google.com/github/FelipeSrojas96/DiagnosticoCapstone/blob/main/E06_19349413-7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reconocimiento de Patrones
[Curso de Reconocimiento de Patrones](https://https://domingomery.ing.puc.cl/teaching/patrones/)

Departamento de Ciencia de la Computación

Universidad Catolica de Chile

(c) Domingo Mery, http://domingomery.ing.uc.cl



# Ejercicio 06: Selección de Características para la Detección de espinas de salmón

En este ejercicio se realizará la clasificación entre dos clases de "parches" de radiogarfías de filetes de salmón de 100x100 pixeles (en escala de grises):

* Clase 0: Espinas (contiene parches de filetes de salmón con espinas). [Ver imágenes](https://www.dropbox.com/s/8bbb7yzyrjkl0qs/fishbones_0.png?dl=00)

* Clase 1: No-Espinas (contiene parches de filetes de salmón sin espinas) [Ver imágenes](https://www.dropbox.com/s/tdgp3shvk47213c/fishbones_1.png?dl=0)

La base de datos contiene 680 imágenes de 100x100 pixeles (340 imágenes por clase).

**ADVERTENCIA:** Este ejercicio tiene fines pedagógicos sólamente, con la idea de que la solución a este problema pueda ejecutarse en un par de minutos. Un buen detector de espinas sigue esta idea pero con miles de radiografías por clase.




# Setup inicial

Liberías necesarias para que funcione el algoritmo.

## Instalación de PyBalu

[PyBalu](https://github.com/mbucchi/pybalu) es una librería creada para extraer características. 

In [6]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

from IPython.display import clear_output
!pip install scipy==1.2
!pip3 install pybalu==0.2.5
!pip install passlib
clear_output()
print('Librerías instaladas.')

Librerías instaladas.


## Setup de librerías

In [7]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#
import numpy as np
import matplotlib.pyplot as plt
import os.path
from   sklearn.metrics import confusion_matrix, accuracy_score
from   sklearn.neighbors import KNeighborsClassifier
from   tqdm.auto import tqdm
from   pybalu.feature_extraction import lbp_features
from   pybalu.feature_transformation import normalize
from   pybalu.feature_analysis import jfisher
clear_output()
print('Librerías cargadas.')



Librerías cargadas.


## Funciones necesarias

In [8]:
#
# EJECUTAR ESTA CELDA SIN MODIFICARLA
#

def num2fixstr(x,d):
  # example num2fixstr(2,5) returns '00002'
  # example num2fixstr(19,3) returns '019'
  st = '%0*d' % (d,x)
  return st

def ImageLoad(prefix,num_char,num_img,echo='off'):
  st   = prefix + num2fixstr(num_char,2) + '_' + num2fixstr(num_img,4) + '.png'
  if echo == 'on':
    print('loading image '+st+'...')
  img    = 255*plt.imread(st)
  return img

# Separación entre training y testing
def SplitTrainTest(X,y,n):

  K      = np.max(y)+1              # número de clases
  N      = np.int(X.shape[0]/K)     # numeros de muestras por clase
  Ntrain = n*K                      # número de muestras para el training
  Ntest  = K*N-Ntrain               # número de muestras para el testing
  M      = X.shape[1]               # número de características por muestra
  Xtrain = np.zeros((Ntrain,M))     # subset de training
  ytrain = np.zeros((Ntrain),'int') # ground truth del training         
  Xtest  = np.zeros((Ntest,M))      # subset de testing
  ytest  = np.zeros((Ntest),'int')  # ground truth del testing  

  # contadores
  itrain = 0
  itest  = 0
  t      = 0

  for j in range(K):     # para cada clase
    for i in range(N):   # para cada imagen de la clase
      if i<n: # training
        Xtrain[itrain,:] = X[t,:]
        ytrain[itrain] = y[t]
        itrain = itrain+1
      else:  # testing
        Xtest[itest,:] = X[t,:]
        ytest[itest] = y[t]
        itest = itest+1
      t = t+1
  
  return Xtrain,ytrain,Xtest,ytest

# Clasificación usando KNN con 25 vecinos
def ClassifierKNN(Xtrain,ytrain,Xtest,ytest,echo ='on'):
  knn = KNeighborsClassifier(n_neighbors=25)
  knn.fit(Xtrain, ytrain)
  ypred        = knn.predict(Xtest)
  acc          = accuracy_score(ytest,ypred)
  C = confusion_matrix(ytest,ypred)
  if echo == 'on':
    print('Entrenando con '+str(Xtrain.shape[0])+' muestras y probando con '+str(Xtest.shape[0])+' muestras')
    print('Los datos tienen '+str(Xtrain.shape[1])+' features')
    print('Testing Accuracy = '+str(acc*100)+'%')
    print('Matriz de Confusión:')
    print(C)
  return acc,C

## ADVERTENCIA

<font color='red'> PARA LA EJECUCIÓN DE ESTA TAREA NO DEBE USAR OTRAS LIBRERÍAS 

(sólo está permitido usar las librerías definidas en las celdas anteriores)

# PREGUNTA 1: Carga de base de datos

(1 punto)

La base de datos consiste en 2 clases con 340 imágenes por clase. Se almacenan en la carpeta `fishbones` con el formato `fish_xx_nnnn.png`, donde `xx` es el ID de la clase (00 para espinas, 02 para no-espinas) y `nnnn` es el número de la foto de la clase (0001, 0002, ... 0340). Las radiografías son de 100x100 pixeles en escala de grises.


En esta celda debes cargar la base de datos a este ambiente de trabajo, descargando el archivo fishbones.zip que se encuentra en este link: `https://www.dropbox.com/s/7d9y6kllguegk77/fishbones.zip`. Una vez descargado el archivo zip, debes descomprimirlo.

AYUDA: usar comando `!wget <url archivo zip>` para descargar el archivo, y el comando `!unzip <archivo zip>` para descomprimirlo.



In [9]:
# Carga de base de datos

# COMPLETAR AQUI DOS LINEAS DE CODIGO
# Linea 1: comando para bajar archivo zip desde dropbox
# Linea 2: comando para descomprimir el archivo zip bajado en la linea 1
# Al final de la ejecucion debe haberse creado la carpeta fishbones con 680 imágenes

!wget https://www.dropbox.com/s/7d9y6kllguegk77/fishbones.zip # <- COMPLETAR AQUI LINEA 1
!unzip -qq fishbones.zip # <- COMPLETAR AQUI LINEA 2


--2022-04-20 22:10:56--  https://www.dropbox.com/s/7d9y6kllguegk77/fishbones.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7d9y6kllguegk77/fishbones.zip [following]
--2022-04-20 22:10:56--  https://www.dropbox.com/s/raw/7d9y6kllguegk77/fishbones.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc05639801514a137ef0450b31e2.dl.dropboxusercontent.com/cd/0/inline/Bjx3oSxY4iHLQwF9p0s-fJnZEHnfzUl9FnGUagNaamW2E8HpP5qshCBtDfRwacdF3JwCLB3fuEwxU5OQfHFqJq_7qc43Vrxxd_JwQNujOWNdiFZFfecN33i9BNylQ4XAd9SCCnulbSUTvBaNOfdt58OPvbaZZn2YeG8dApc4eK552Q/file# [following]
--2022-04-20 22:10:56--  https://uc05639801514a137ef0450b31e2.dl.dropboxusercontent.com/cd/0/inline/Bjx3oSxY4iHLQwF9p0s-fJnZEHnfzUl9FnGUagNaamW2E8HpP5qshCBtDfRwacdF3

# PREGUNTA 2: Extracción de características

(2 puntos)

En esta pregunta se debe realizar la extracción de características de la imagen `img`. Las características a extraer son las basadas en LBP (invariantes a la rotación) y se extraen usando los siguientes comandos de la librería PyBalu:

## Cómo extraer características LBP rotación-invairiante:

Con el comando

`f  = lbp_features(img, hdiv=a, vdiv=b, mapping='uniform')`

se divide la imagen `img` en `a x b` particiones y en cada una de ellas se extrae el descriptor LBP (vector de 10 elementos). En este caso `f` es un vector de `10*a*b` elementos. En este ejercicio trabajaremos con `a = b = 1`, es decir sólo una partición por imagen.



<font color='orange'> IMPORTANTE: 

La salida de esta celda debe ser:

* `Xlbp`: una matriz de 680 x 10 elementos. Las primeras 340 filas corresponden a los LBP de las imágenes de la clase 0, mientras que las siguientes 340 filas corresponden a los LBP de las imágenes de la clase 1.

* `y`: vector con el Ground Truth de cada muestra. Este vector debe ser un array del tipo `int` con 680 elementos (340 ceros y 340 unos). 

<font color='red'> Para escribir este código reutilice los códigos vistos en clase


In [19]:
#
# COMPLETAR CODIGO DE LA PREGUNTA 2 AQUI:
#
Xlbp = []

for i in range(1,681):
  if i>340:
    clase = 1
    i -= 340
  else: 
    clase = 0
  img = ImageLoad('fishbones/fish_',clase,i)
  x_lbp = f = lbp_features(img, hdiv=1, vdiv=1, mapping='uniform')
  Xlbp.append(x_lbp)

y1 = np.zeros(340,dtype=int)
y2 = np.ones(340,dtype = int)
y = np.concatenate((y1,y2),axis = 0)

Xlbp = np.array(Xlbp)

print(Xlbp.shape)




(680, 10)


# Pregunta 3: Clasificación con todas las características

(1 punto)

En todos los experimentos usaremos 280 muestras por clase para training y el resto (60) para testing.

Como primer experimento clasificaremos usando todas las características extraídas, es decir con 10 características. Para la separación train/test usaremos la función `SplitTrainTest`. Para la clasificación usaremos la función `ClassifierKNN`.

El output de esta celda debe ser:

* `Xtrain,ytrain,Xtest,ytest`: Datos del training y testing

* `acc`: Accuracy obtenido

* `C`: Matriz de confusión

<font color='orange'>Se recomienda estudiar bien las dos líneas de código que hacen la normalización.</font>





In [20]:
# Definición de datos de entrada
X = Xlbp                         # <= COMPLETAR AQUI LA MATRIZ DE LAS CARACTERISTICAS LBP EXTRAIDAS

# Separación de datos de Training/Testing
ntrain = 280                     # <= COMPLETAR AQUI EL NUMERO DE MUESTRAS POR CLASE PARA EL TRAINING
Xtrain,ytrain,Xtest,ytest = SplitTrainTest(X,y,ntrain)  # <= COMPLETAR AQUI LA SEPARACIÓN TRAINING/TESTING

# NORMALIZACION: ESTUDIAR PASO 1 y PASO 2
# Normalización (paso 1): cada columna del training es escalada a numeros entre 0 y 1
Xtrain, a, b = normalize(Xtrain)

# Normalización (paso 2): cada columna del testing es escalda usando el escalamiento del training
# Observar que los factores de escalamiento (a,b) dependen de los datos del training y no del testing.
Xtest        = Xtest * a + b

acc,C = ClassifierKNN(Xtrain,ytrain,Xtest,ytest)    # <= COMPLETAR AQUI LA LÍNEA DE CÓDIGO QUE REALIZA LA CLASIFICACIÓN


Entrenando con 560 muestras y probando con 120 muestras
Los datos tienen 10 features
Testing Accuracy = 75.0%
Matriz de Confusión:
[[41 19]
 [11 49]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


# PREGUNTA 4: Selección de características usando "Búsqueda Exhaustiva"

(8 puntos)

En este ejemplo buscaremos las q=4 mejores características de las m=10 extraídas. La búsqueda exhaustiva -en este caso- evalúa todas las combinaciones de 4 características (sin repetición), es decir, evalúa las siguientes m!/(q! x (m-q!) = 10!/(4! x (10-4)!) = 210 combinaciones:

* Combinación 1: (0,1,2,<font color='orange'> 3 </font> )

* Combinación 2: (0,1,2,<font color='orange'> 4 </font>)

* Combinación 3: (0,1,2,<font color='orange'> 5 </font>)

* Combinación 4: (0,1,2,<font color='orange'> 6 </font>)

* Combinación 5: (0,1,2,<font color='orange'> 7 </font>)

* Combinación 6: (0,1,2,<font color='orange'> 8 </font>)

* Combinación 7: (0,1,2,<font color='orange'> 9 </font>)

* Combinación 8: (0,1,<font color='magenta'> 3 </font>,<font color='cyan'> 4 </font>) -> la combinación (0,1,3,0) no se evalúa porque se repite el 0, además la combinación (0,1,3,2) tampoco se evalúa porque ya fue evaluada en la combinación 1 (en otro orden).

* Combinación 9: (0,1,<font color='magenta'> 3 </font>,<font color='cyan'> 5 </font>)

* Combinación 10: (0,1,<font color='magenta'> 3 </font>,<font color='cyan'> 6 </font>)

* :

* Combinación 210: (6,7,8,9)

La búsqueda exhaustiva escoge la "mejor" de estas 210 combinaciones. La mejor combinación será aquella que tenga la mayor "separabilidad" entre las clases. Para obtener un "score" de la separabilidad de las características podemos usar la función de Fisher que mide qué tan compactas están cada una de las clases, y qué tan separadas están entre ellas. Una buena separabilidad (un buen "score") se obtiene si en el espacio de características, las clases son compactas y la separación entre ellas es grande. Este "score" está implementado en la función `jfisher`. Entre más grande el "score" mayor es la separabilidad. En la búsqueda exhaustiva buscamos aquella combinación que maximiza el score entregado por `jfisher`.

## Ejemplo de cómo usar `jfisher`:

A manera de ejemplo, vamos a suponer que hemos almacenado en `Z` la extracción de 20 características de 1000 muestras, es decir el tamaño de esta matriz `Z` es de 1000x20. Adicionalmente, la clasificación ideal de estas 1000 muestras se encuentra almacenada en la variable `d` de 1000 elementos. En este ejemplo, los elementos de `d` toman los valores 0, 1, 2 ó 3 (porque hay 4 clases).

En este ejemplo, deseamos evaluar el "score" de las características (3,15,17). Para evaluar el desempeño de estas tres características seguimos estos cuatro pasos:

* Definimos `i1=3`, `i2=15`, y `i3=17`.

* Definimos `ii = (i1,i2,i3)`.

* Escogemos de `Z` las columnas indexadas por `ii` usando `Z_sel = Z[:,ii]`. Ahora `Z_sel` tiene 1000 muestras y 3 columnas. 

* Para obtener el "score" de estas características seleccionadas, usamos el comando: `score = jfisher(Z_sel,d)`

En este ejemplo, en la variable `score` hemos almacenado la evaluación de la función de Fisher de la separabilidad para la selección de las columas (3,15,17).

En el siguiente código Ud. debe implementar la búsqueda exhaustiva. El input de este código es la matriz `Xtrain` de 640x10 elementos, y el vector `ytrain` de 640 elementos, la salida debe ser el vector `best_indices` de 4 números distintos entre 0 y 9 (ya que existen 10 características en `Xtrain` y se escogen 4), que corresponde a la mejor combinación de características. 

En este código Ud. debe hacer  210 evaluaciones de la función de Fisher, una para cada combinación, y se selecciona aquella combinación que maximiza la función de Fisher.

La búsqueda exhaustiva se realiza con cuatro contadores `(i1,i2,i3,i4)` para cada una de las 4 características. Como nuestros datos tienen 10 características, es necesario observar que 

* `i1` debe contar de 0 a 6

* `i2` debe contar de `i1+1` a 7

* `i3` debe contar de `i2+1` a 8

* `i4` debe contar de `i3+1` a 9


En esta implemantación, `i1 < i2 < i3 < i4 `, de esta manera 1) se recorren todos los índices sin repetirse, es decir la combinación (1,1,2,3) no debe evaluarse, y 2) no se evalúan combinaciones que ya han sido evaluadas anteriormente, es decir si evaluó (1,2,3,4), no se evalúa (3,2,1,4) ya que ambas combinaciones tienen el mismo "score".



In [34]:
#######################################################################
# AYUDA: Hay que leer y entender el texto anterior (de la Pregunta 4) #
#######################################################################

Z = Xtrain
d = ytrain
max_score = None 
best_indices = None
#
# COMPLETAR CODIGO DE LA PREGUNTA 4 AQUI:
#
score = 0
for i in range(7):
  for j in range(i+1,8):
    for k in range(j+1,9):
      for l in range(k+1,10):
        score += 1
        ii = (i,j,k,l)
        Z_sel = Z[:,ii]
        score = jfisher(Z_sel,d)
        if(max_score is None or score >max_score):
          max_score = score 
          best_indices = ii


print('Caractersiticas Seleccionadas: ',best_indices) # <= Ultima linea, no modificar


Caractersiticas Seleccionadas:  (2, 5, 6, 9)


# PREGUNTA 5: Clasificación con características seleccionadas

(3 puntos)

A continuación se definen las matrices `Xtrain_sel` (de 640x4 elementos) y `Xtest_sel` (de 40x4) elementos, tomando las columnas seleccionadas de `Xtrain` y `Xtest` respectivamente usando los índices encontrados en `best_indices`.

Finalmente, entrenamos y probamos el clasificador con estos nuevos datos. No es necesario definir `ytrain_sel` y `ytest_sel` porque son iguales a `ytrain` y `ytest` respectivamente.

El output de esta celda debe ser:

* `acc`: Accuracy obtenido

* `C`: Matriz de confusión


In [35]:
#
# COMPLETAR CODIGO DE LA PREGUNTA 5 AQUI:
#
selected = (2,5,6,9)
Xtrain_sel = Xtrain[:,selected] 
Xtest_sel = Xtest[:,selected]

print(Xtrain_sel.shape)
print(Xtest_sel.shape)
# NORMALIZACION: ESTUDIAR PASO 1 y PASO 2
# Normalización (paso 1): cada columna del training es escalada a numeros entre 0 y 1
#Xtrain_sel, a, b = normalize(Xtrain_sel)

# Normalización (paso 2): cada columna del testing es escalda usando el escalamiento del training
# Observar que los factores de escalamiento (a,b) dependen de los datos del training y no del testing.
#Xtest        = Xtest * a + b

acc,C = ClassifierKNN(Xtrain_sel,ytrain,Xtest_sel,ytest)    # <= COMPLETAR AQUI LA LÍNEA DE CÓDIGO QUE REALIZA LA CLASIFICACIÓN


(560, 4)
(120, 4)
Entrenando con 560 muestras y probando con 120 muestras
Los datos tienen 4 features
Testing Accuracy = 82.5%
Matriz de Confusión:
[[46 14]
 [ 7 53]]


<font color='orange'> APRENDIZAJE: Si has hecho los pasos correctamente, observarás que con 4 características se obtienen en este problema un mejor desempeño que con 10 características. En esta solución, con selección de características, hemos subido el accuracy en 7.5%.


# VERIFICACION FINAL

<font color='red'> **ADVERTENCIA:** Este ejercicio será evaluado de la siguiente manera: si el código funciona y el resultado es correcto, la pregunta tendrá un 100% de la evaluación, en caso contrario 0%. Para asegurarse que el código se ejecute sin caídas seleccione la opción del menú:

1) <font color='orange'> 'Runtime' + 'Factory reset runtime'

2) <font color='orange'> 'Run all'

<font color='red'> El código debería ejecutarse desde el inicio hasta el final sin problema alguno.